Instalar dependencias

In [1]:
!pip install sagemaker_datawrangler

  Using cached sagemaker_datawrangler-0.4.3-py3-none-any.whl.metadata (667 bytes)
  Using cached sagemaker_data_insights-0.4.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached ipywidgets-7.8.5-py2.py3-none-any.whl.metadata (1.9 kB)
  Using cached sagemaker_scikit_learn_extension-2.5.0-py3-none-any.whl
  Using cached ipython_genutils-0.2.0-py2.py3-none-any.whl.metadata (755 bytes)
  Using cached widgetsnbextension-3.6.10-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached jupyterlab_widgets-1.1.11-py3-none-any.whl.metadata (3.7 kB)
Using cached sagemaker_datawrangler-0.4.3-py3-none-any.whl (10.6 MB)
Using cached sagemaker_data_insights-0.4.0-py3-none-any.whl (99 kB)
Using cached ipywidgets-7.8.5-py2.py3-none-any.whl (124 kB)
Using cached ipython_genutils-0.2.0-py2.py3-none-any.whl (26 kB)
Using cached jupyterlab_widgets-1.1.11-py3-none-any.whl (246 kB)
Using cached widgetsnbextension-3.6.10-py2.py3-none-any.whl (1.6 MB)
  Attempting uninstall: jupyterlab-widgets
    Found existing in

Preparación

In [137]:
import sagemaker_datawrangler           # For interactive data prep widget
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker

In [138]:
import sagemaker
bucket=sagemaker.Session().default_bucket()

In [139]:
bucket # Bucket que se crea cuando creamos nuestro Dominio e iniciaos el Studio Een SageMaker

'sagemaker-us-east-2-244680720635'

In [140]:
prefix = 'mlops/tarea2'

In [141]:
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

In [142]:
role

'arn:aws:iam::244680720635:role/service-role/AmazonSageMaker-ExecutionRole-20250331T182939'

Data

In [38]:
!wget https://raw.githubusercontent.com/velasquezjeisson/UdeMCloud/refs/heads/main/Telco-Customer-Churn.csv --no-check-certificate

--2025-04-04 21:34:19--  https://raw.githubusercontent.com/velasquezjeisson/UdeMCloud/refs/heads/main/Telco-Customer-Churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
  Unable to locally verify the issuer's authority.
200 OKequest sent, awaiting response... 
Length: 977501 (955K) [text/plain]
Saving to: ‘Telco-Customer-Churn.csv’

Telco-Customer-Chur 100%[===================>] 954.59K  --.-KB/s    in 0.006s  

2025-04-04 21:34:19 (146 MB/s) - ‘Telco-Customer-Churn.csv’ saved [977501/977501]



In [18]:
#!pip install ipywidgets 


In [143]:
df = pd.read_csv('Telco-Customer-Churn.csv', delimiter=',', header=0)
pd.set_option('display.max_columns', None)
# df['TotalCharges'] = df['TotalCharges'].str.replace(' ', np.nan, regex=True)
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
df.dropna(inplace=True)
#df.set_index('customerID', inplace=True)

df.head()


   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity OnlineBackup  \
0  No phone service             DSL             No          Yes   
1                No             DSL            Yes           No   
2                No             DSL            Yes          Yes   
3  No phone service             DSL            Yes           No   
4                No     Fiber optic             No           No   

  DeviceProtection TechSupport StreamingTV StreamingMovies        Contract  \
0               No          No        

Descargar Script

In [31]:
!wget https://raw.githubusercontent.com/velasquezjeisson/UdeMCloud/refs/heads/main/ScriptTarea2.py --no-check-certificate

--2025-04-04 20:30:29--  https://raw.githubusercontent.com/velasquezjeisson/UdeMCloud/refs/heads/main/ScriptTarea2.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
  Unable to locally verify the issuer's authority.
200 OKequest sent, awaiting response... 
Length: 1619 (1.6K) [text/plain]
Saving to: ‘ScriptTarea2.py’

ScriptTarea2.py     100%[===================>]   1.58K  --.-KB/s    in 0s      

2025-04-04 20:30:29 (13.9 MB/s) - ‘ScriptTarea2.py’ saved [1619/1619]



Subir a Bucket

In [144]:
# Upload to S3 Bucket
from sagemaker import Session
prefix = 'mlops/tarea2'

sess = Session()
input_source = sess.upload_data('./Telco-Customer-Churn.csv', bucket=bucket, key_prefix=f'{prefix}/input_data')
input_source

's3://sagemaker-us-east-2-244680720635/mlops/tarea2/input_data/Telco-Customer-Churn.csv'

In [145]:
role = get_execution_role()

In [146]:
role

'arn:aws:iam::244680720635:role/service-role/AmazonSageMaker-ExecutionRole-20250331T182939'

In [147]:
train_path = f"s3://{bucket}/{prefix}/train"
validation_path = f"s3://{bucket}/{prefix}/validation"
test_path = f"s3://{bucket}/{prefix}/test"

In [148]:
test_path

's3://sagemaker-us-east-2-244680720635/mlops/tarea2/test'

Preprocesamiento de datos

In [165]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role

# Configurar el procesador
sklearn_processor = SKLearnProcessor(
    framework_version='1.2-1',
    role=get_execution_role(),
    instance_type='ml.t3.medium',
    instance_count=1
)

# Ejecutar el procesador
sklearn_processor.run(
    code='ScriptTarea2.py',
    inputs=[
        ProcessingInput(
            source=input_source,
            destination="/opt/ml/processing/input",
            s3_input_mode="File",
            s3_data_distribution_type="ShardedByS3Key"
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="train_data",
            source="/opt/ml/processing/output/train",
            destination=train_path,
        ),
        ProcessingOutput(output_name="validation_data", source="/opt/ml/processing/output/validation", destination=validation_path),
        ProcessingOutput(output_name="test_data", source="/opt/ml/processing/output/test", destination=test_path),
    ],
    arguments=[
        '--input_file', '/opt/ml/processing/input/Telco-Customer-Churn.csv',
        '--train_output', '/opt/ml/processing/output/train',
        '--validation_output', '/opt/ml/processing/output/validation',
        '--test_output', '/opt/ml/processing/output/test'
    ]
)

..............................
.

Imagen de Sklearn

In [166]:
sklearn_processor.image_uri

'257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3'

In [167]:
!aws s3 ls $train_path/

2025-04-05 05:20:35     757463 train.csv


In [168]:
!aws s3 ls $test_path/

2025-04-05 05:20:35     249899 X_test.csv
2025-04-05 05:06:17     252713 test.csv
2025-04-05 05:20:35       2814 y_test.csv


In [169]:
!aws s3 ls $validation_path/

2025-04-05 05:20:35     252665 validation.csv


Build del modelo

In [170]:
role = get_execution_role()

bucket=sagemaker.Session().default_bucket()
prefix = 'mlops/tarea2'
sess = Session()
train_path = f"s3://{bucket}/{prefix}/train"
validation_path = f"s3://{bucket}/{prefix}/validation"
test_path = f"s3://{bucket}/{prefix}/test"

In [171]:
container = sagemaker.image_uris.retrieve(
    framework='xgboost',
    region=boto3.Session().region_name,
    version='latest'
)


In [172]:
container

'825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest'

In [173]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

In [174]:
s3_input_train.config

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
   'S3Uri': 's3://sagemaker-us-east-2-244680720635/mlops/tarea2/train',
   'S3DataDistributionType': 'FullyReplicated'}},
 'ContentType': 'csv'}

In [175]:
s3_input_validation.config

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
   'S3Uri': 's3://sagemaker-us-east-2-244680720635/mlops/tarea2/validation/',
   'S3DataDistributionType': 'FullyReplicated'}},
 'ContentType': 'csv'}

In [176]:
xgb = sagemaker.estimator.Estimator(container,
                                    role,
                                    instance_count=1,
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(
            max_depth=5,
            eta=0.2,
            gamma=4,
            min_child_weight=6,
            subsample=0.8,
            objective='binary:logistic',
            num_round=1000,
            eval_metric='auc',
            delimiter=','
        )

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) #Let's to see logs in training jobs

2025-04-05 05:23:31 Starting - Starting the training job...
..25-04-05 05:24:00 Starting - Preparing the instances for training.
..25-04-05 05:24:24 Downloading - Downloading input data.
.....04-05 05:24:49 Downloading - Downloading the training image.
Arguments: training - Training image download completed. Training in progress..
[2025-04-05:05:26:01:INFO] Running standalone xgboost training.
[2025-04-05:05:26:01:INFO] File size need to be processed in the node: 0.96mb. Available memory size in the node: 8555.15mb
[2025-04-05:05:26:01:INFO] Determined delimiter of CSV input is ','
[05:26:01] S3DistributionType set as FullyReplicated
[05:26:01] 4219x29 matrix with 122351 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2025-04-05:05:26:01:INFO] Determined delimiter of CSV input is ','
[05:26:01] S3DistributionType set as FullyReplicated
[05:26:01] 1406x29 matrix with 40774 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&del

Interfaz de modelo

In [177]:
test_path

's3://sagemaker-us-east-2-244680720635/mlops/tarea2/test'

In [178]:
!aws s3 ls $test_path

                           PRE test/


In [179]:
test_data_x = pd.read_csv(os.path.join(test_path, 'X_test.csv'),header=None)
test_data_y = pd.read_csv(os.path.join(test_path, 'y_test.csv'),header=None)

Endpoint

In [180]:
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge') #Abrir end--points y logs

------!

In [183]:
#Evaluation
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()

In [184]:
xgb_predictor.serializer

In [185]:

def predict(data, predictor, rows=500):
    """
    Make predictions using a SageMaker predictor.

    Args:
        data (pd.DataFrame): Input data for predictions.
        predictor (SageMaker Predictor): Predictor for making predictions.
        rows (int): Number of rows per batch for prediction.

    Returns:
        np.ndarray: A NumPy array containing the predictions.
    """
    # Split the data into batches based on the specified number of rows
    split_array = np.array_split(data, int(np.ceil(data.shape[0] / float(rows))))

    # Initialize a list to store predictions
    predictions = []

    # Make predictions for each batch of data
    for array in split_array:
        # Make the prediction and decode the result
        batch_predictions = predictor.predict(array).decode('utf-8')
        # Append predictions to the list
        predictions.append(batch_predictions)

    # Join all predictions into a single string and convert to a NumPy array
    predictions_string = ','.join(predictions)  # Join all predictions into a single string
    return np.fromstring(predictions_string, sep=',')  # Convert to a NumPy array


In [186]:
# Call the function to get predictions
predictions = predict(test_data_x, xgb_predictor)

# Optional: Display the predictions
print("Predictions:", predictions)

Predictions: [0.07247873 0.72890872 0.8852275  ... 0.32138747 0.46760577 0.59564   ]


In [188]:
confusion =pd.crosstab(index=test_data_y[0], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])
print(confusion)


predictions  0.0  1.0
actuals              
0            917  126
1            192  172


Delete Endpoint

In [189]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)